# Hillsborough County Parcel Comp Analysis

Placeholder for some description

Setup the workspace

In [2]:
import geopandas as gpd
import pandas as pd

Define paths for ETL

In [13]:
sales_path = '../data/raw/allsales_03_07_2025/allsales.dbf'
parcel_path = '../data/raw/parcel_03_07_2025/parcel.dbf'
sub_path = '../data/raw/parcel_03_07_2025/parcel_sub_names.dbf'
dor_path = '../data/raw/parcel_03_07_2025/parcel_dor_names.dbf'

Load the data into DataFrames

In [15]:
df_parcel = gpd.read_file(parcel_path)
df_sales = gpd.read_file(sales_path, encoding='ISO-8859-1')
df_sub = gpd.read_file(sub_path)[['SUBCODE', 'SUBNAME']].drop_duplicates()
df_dor = gpd.read_file(dor_path)

Export the data to parquet

In [18]:
df_sales.to_parquet('../data/processed/allsales.parquet', engine = 'pyarrow')

In [19]:
# Add DOR and SUB descriptive names to parcel data
df_merge = df_parcel.merge(df_dor, left_on = 'DOR_C', right_on='DORCODE', how ='left').drop(columns = ['DOR_C'])
df_merge = df_merge.merge(df_sub, left_on ='SUB', right_on='SUBCODE', how ='left').drop(columns = ['SUB'])

df_parcel.to_parquet('../data/processed/parcel.parquet', engine = 'pyarrow')

Read in parquet files

In [20]:
df_sales = pd.read_parquet('../data/processed/allsales.parquet')
df_parcel = pd.read_parquet('../data/processed/parcel.parquet')

In [7]:
df = pd.merge(df_sales, df_parcel, on='FOLIO', how='left')